In [1]:
import json
import os
import pandas as pd
import numpy as np
from datetime import datetime 
import matplotlib.pyplot as plt
import re
from matplotlib.pyplot import figure

In [2]:
output_dir = "error_processed_data/"
file = "error_tree_2.pki"
raw_data = pd.read_pickle(output_dir + file)
raw_data

,time,User Name,Machine ID,command,full_command,stderr,stdout
0,2021-04-19 19:34:00,Amy,10bcd15c,ls,ls,,SuperShell\nstderr.txt
1,2021-04-19 19:35:00,Amy,10bcd15c,export,export $TEST=Hello,/home/supershell/.offline.sh: line 358: export...,
2,2021-04-19 19:35:00,Amy,10bcd15c,export,export $TEST='Hello',/home/supershell/.offline.sh: line 358: export...,
3,2021-04-19 19:36:00,Amy,10bcd15c,export,export PATH=$PATH:$HOME/bin,,
4,2021-04-19 19:36:00,Amy,10bcd15c,export,export PATH=$PATH:test/bin,,
...,...,...,...,...,...,...,...
1065,2023-01-19 14:33:00,Jane,a5cc4,ls,ls,,
1066,2023-01-19 14:33:00,Jane,a5cc4,pwd,pwd,,/home/jovyan/work/Bash
1067,2023-01-19 14:34:00,Jane,a5cc4,pwd,pwd,,/home/jovyan/work/Bash
1068,2023-01-19 14:39:00,Jane,a5cc4,ls,ls ~/data-store/data/input/Discovery/TSVFiles/,,


In [3]:
full_history = raw_data[['time', 'User Name', 'command', 'full_command']]
full_history['No Command Error'] = 'None'
full_history['Error Type'] = 'None'
full_history.head()

<ipython-input-3-9ee16d08db0e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_history['No Command Error'] = 'None'
<ipython-input-3-9ee16d08db0e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_history['Error Type'] = 'None'


,time,User Name,command,full_command,No Command Error,Error Type
0,2021-04-19 19:34:00,Amy,ls,ls,None,None
1,2021-04-19 19:35:00,Amy,export,export $TEST=Hello,None,None
2,2021-04-19 19:35:00,Amy,export,export $TEST='Hello',None,None
3,2021-04-19 19:36:00,Amy,export,export PATH=$PATH:$HOME/bin,None,None
4,2021-04-19 19:36:00,Amy,export,export PATH=$PATH:test/bin,None,None


**Analyze on preceding commands of error**

In [5]:
output_dir = "error_processed_data/"
file = "only_error.pki"
error_data = pd.read_pickle(output_dir + file)
error_data

,time,User Name,command,full_command,error,No Command Error,Error Type
1,2021-04-19 19:35:00,User_1,export,export $TEST=Hello,PATH: $: not a valid identifier,PATH: $: not a valid identifier,Other Error
2,2021-04-19 19:35:00,User_1,export,export $TEST='Hello',PATH: $: not a valid identifier,PATH: $: not a valid identifier,Other Error
18,2022-09-11 18:27:00,User_3,ls,ls /home/jovyan/RNA_SEQ_SAMPLES,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
22,2022-09-11 18:28:00,User_3,ls,ls /gunings/RNA_SEQ_SAMPLES,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
29,2022-09-11 18:30:00,User_3,cd,cd sdgeorge,PATH: $: No such file or directory,PATH: $: No such file or directory,Argument Name Error
...,...,...,...,...,...,...,...
754,2022-12-01 20:11:00,Adam,ls,ls –l data/input/Discovery,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
758,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
759,2022-12-01 20:17:00,Adam,ls,ls –l ./data/input/RNA_SEQ_SAMPLES stderr.txt,ls: $: No such file or directory,ls: $: No such file or directory,Argument Name Error
779,2022-12-01 20:23:00,Adam,cd,cd ~/data/input/RNA_SEQ_SAMPLES/Dcd ~/data/inp...,PATH: $: too many arguments,PATH: $: too many arguments,Command Argument Clash Error


In [10]:
full_history.loc[error_data.index] = error_data

/Users/guningshen/Library/Python/3.9/lib/python/site-packages/pandas/core/indexing.py:1787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


In [11]:
full_history

,time,User Name,command,full_command,No Command Error,Error Type
0,2021-04-19 19:34:00,User_1,ls,ls,None,None
1,2021-04-19 19:35:00,User_1,export,export $TEST=Hello,PATH: $: not a valid identifier,Other Error
2,2021-04-19 19:35:00,User_1,export,export $TEST='Hello',PATH: $: not a valid identifier,Other Error
3,2021-04-19 19:36:00,User_1,export,export PATH=$PATH:$HOME/bin,None,None
4,2021-04-19 19:36:00,User_1,export,export PATH=$PATH:test/bin,None,None
...,...,...,...,...,...,...
814,2023-01-19 14:40:00,Lily,touch,touch ~/data-store/data/input/Discovery/TSVFil...,touch: $: Permission denied,Permission Error
815,2022-12-15 19:11:00,Megan,clear,clear,None,None
816,2022-12-15 19:11:00,Megan,ls,ls,None,None
817,2022-12-15 19:14:00,Megan,cd,cd,None,None


**Change full command file and directory name**

In [ ]:
file_name_replacer = re.compile(r'.*((\')|(\")).+((\')|(\")).*')
file_name = re.compile(r'((\')|(\")).+((\')|(\"))')

path_replacer = re.compile(r'(\/).+(: line )(\d+).*')
path_name = re.compile(r'(\/).+(: line )(\d+)')

name_between_colon = re.compile(r'.+(:).+(:).+')
colon_name = re.compile(r'(:).+(:)')

**Get Command History around each error**

In [13]:
error_graph = full_history.copy()

My Idea is to create a list of 5 commands history for each error to extract the workflow

In [38]:
# function to parse error window into list of 7 full command history
def get_command_window(full_command_list, short_command_list, error_list):
    seven_command_collection = [];
    for i in range(len(full_command_list)):
        if (error_list[i] != "None"):
            w = []
            for j in range(i, i+4):
                if (j >= 0 | j < len(full_command_list)):
                    if (j == i):
                        command_tuple = tuple((full_command_list[j], short_command_list[j], error_list[i]))
                    else:
                        command_tuple = tuple((full_command_list[j], short_command_list[j], 'None'))
                    w.append(command_tuple);
            
            seven_command_collection.append(w)
    
    return seven_command_collection

In [39]:
# get a list of all command history
full_command_list = error_graph["full_command"].to_list()
command_list = error_graph["command"].to_list()
error_list = error_graph["Error Type"].to_list()

# get a list of 3 full command before and after each error
error_window = get_command_window(full_command_list, command_list, error_list)

In [40]:
error_window[16]

[('cd /home/gunings/RNA_SEQ_SAMPLES/Drug_B', 'cd', 'Argument Name Error'),
 ('ls', 'ls', 'None'),
 ('cd home', 'cd', 'None'),
 ('ls', 'ls', 'None')]

**Construct the history of command into a graph**

In [42]:
class Node:

    def __init__(self, program, frequency = 1, error='None') -> None:
        ''' 
        Args:
            program (String): Program that Node represents
            frequency (int): amount of times in data command proceeds parent_command
           
            command (dict): Full commands with frequencies to be used by parent nodes
            children (dict): dict of children nodes, key command, value Node
        '''
        
        self.program = program
        self.frequency = frequency
        self.error = error
        self.commands = {} # key cmd, value frequency
        self.children = {}

In [43]:
def construct_graph(error_window, error_dict={}):

    cur_node = None
    child_node = None
    
    # First loop gets one command window
    for i in range(len(error_window)):
        
        error = error_window[i][0][2]
        first_cmd = error_window[i][0][1]

        if error_dict.get(error) is None:
            cur_node = Node(program=first_cmd, error=error)
            error_dict[error] = cur_node
        else:
            cur_node = error_dict.get(error)
            cur_node.frequency += 1
        
        if cur_node.commands.get(first_cmd) is None:
            cur_node.commands[first_cmd] = 1
        else:
            cur_node.commands[first_cmd] += 1

        for cmd in range(1, 3):
            program = error_window[i][cmd][1]
            full_program = error_window[i][cmd][0]

            if cur_node.children.get(program) is None:
                child_node = Node(program = program)
                cur_node.children[program] = child_node
            else:
                child_node = cur_node.children.get(program)
                child_node.frequency += 1
            
            if child_node.commands.get(full_program) is None:
                child_node.commands[full_program] = 1
            else:
                child_node.commands[full_program] += 1
            
            cur_node = child_node
        
    return error_dict

In [44]:
error_graph = construct_graph(error_window)

In [50]:
import math
def solution_prediction(error, graph):
    cur_node = None
    child_node = None

    if graph.get(error) is None:
        return 'Not supported';
    else:
        cur_node = graph.get(error)
        
        max_frequency = -math.inf
        next_command = ''
        for child in cur_node.children:
            child_node = cur_node.children[child]
            freq = child_node.frequency
            if freq > max_frequency:
                max_frequency = freq
                next_command = child_node.program

        return next_command

In [51]:
suggestion = solution_prediction('Permission Error', error_graph)
print(suggestion)

ls


In [ ]:
def get_prediction(command_list: list[str], graph, result_size=5, graph_depth=4):
    if len(command_list) == 0:
        return None

    commands = command_list[:graph_depth]
    # should be labeled last command
    last_command = command_list[graph_depth]

    if len(commands) == 0:
        return None, [-1]

    program = commands[0].split()[0]

    frequencies = []
    
    if graph.get(program) is None:
        frequencies.append(-1)
        return None, frequencies
    else:
        node = graph[program]
        frequencies.append(node.frequency)

    for command in commands[1:]:
        program = command.split()[0]
        if node.children.get(program) is not None:
            node = node.children[program]
            frequencies.append(node.frequency)
        else:
            frequencies.append(-1)
            return None, frequencies
    
    # last frequency if it has it
    next_program = last_command.split()[0]

    if node.children.get(next_program) is not None:
        frequencies.append(node.children[next_program].frequency)
    else:
        frequencies.append(-1)
    
    # previous command should be command_list[depth-1]
    return node.get_prediction(previous_command=command_list[graph_depth-1], num_to_return=result_size), frequencies

***Choose cat: cannot access $: No such file or directory as an example***

In [17]:
error_filter = (error_expand["no name"] == "cat: $: No such file or directory")

In [18]:
cat_error = error_expand[error_filter]
cat_error.head()

,time,User Name,command,full_command,stderr,error list,no name
68,2021-04-16 23:13:00,User_9,cat,"cat ""!!!!"" > goodbye.txt",cat: '!!!!': No such file or directory,[cat: '!!!!': No such file or directory],cat: $: No such file or directory
97,2021-04-17 13:50:00,User_9,cat,cat goodbye.txt,cat: goodbye.txt: No such file or directory,[cat: goodbye.txt: No such file or directory],cat: $: No such file or directory
172,2021-04-18 22:05:00,User_4,directory_grep_head_tail,NaN,cat: /home/jovyan/Bash/FilteredSortedOutput/Se...,[/home/jovyan/Bash/bin/directory_grep_head_tai...,cat: $: No such file or directory
176,2021-04-18 22:05:00,User_4,directory_grep_head_tail,NaN,cat: /home/jovyan/Bash/FilteredSortedOutput/Se...,[/home/jovyan/Bash/bin/directory_grep_head_tai...,cat: $: No such file or directory
410,2022-09-11 20:13:00,User_3,cat,cat hello.txt goodbye.txt,cat: goodbye.txt: No such file or directory,[cat: goodbye.txt: No such file or directory],cat: $: No such file or directory


**Cat 1: cat "!!!">goodbye.txt**

In [19]:
cat_error_68 = error_expand.iloc[65:72]
cat_error_68

,time,User Name,command,full_command,stderr,error list,no name
65,2021-04-16 23:13:00,User_9,cat,cat goodbye.txt,,[],None
66,2021-04-16 23:13:00,User_9,cat,cat link_goodbye.txt,,[],None
67,2021-04-16 23:13:00,User_9,echo,"echo ""!!!!"" >> goodbye.txt",,[],None
68,2021-04-16 23:13:00,User_9,cat,"cat ""!!!!"" > goodbye.txt",cat: '!!!!': No such file or directory,[cat: '!!!!': No such file or directory],cat: $: No such file or directory
69,2021-04-16 23:15:00,User_9,cat,cat link_goodbye.txt,,[],None
70,2021-04-16 23:15:00,User_9,pico,NaN,,[],None
71,2021-04-16 23:15:00,User_9,diff,diff goodbye.txt copied_goodbye.txt,,[],None


**Cat 2: cat goodbye.txt**

In [20]:
cat_error_97 = error_expand.iloc[94:101]
cat_error_97

,time,User Name,command,full_command,stderr,error list,no name
94,2021-04-17 13:50:00,User_9,ls,ls -l,,[],None
95,2021-04-17 13:50:00,User_9,mv,mv goodbye.txt renamed_goodbye.txt,,[],None
96,2021-04-17 13:50:00,User_9,ls,ls -l,,[],None
97,2021-04-17 13:50:00,User_9,cat,cat goodbye.txt,cat: goodbye.txt: No such file or directory,[cat: goodbye.txt: No such file or directory],cat: $: No such file or directory
98,2021-04-17 13:57:00,User_9,ls,ls -r,,[],None
99,2021-04-17 13:57:00,User_9,ls,ls -l -r,,[],None
100,2021-04-17 13:57:00,User_9,ls,ls -R,,[],None


Error results because type the wrong file name

**Cat 3: cat hello.txt**

In [21]:
cat_error_410 = error_expand.iloc[407:414]
cat_error_410

,time,User Name,command,full_command,stderr,error list,no name
407,2022-09-11 20:12:00,User_3,pwd,pwd,,[],None
408,2022-09-11 20:13:00,User_3,cat,cat hello.txt goobye.txt,,[],None
409,2022-09-11 20:13:00,User_3,ls,ls,,[],None
410,2022-09-11 20:13:00,User_3,cat,cat hello.txt goodbye.txt,cat: goodbye.txt: No such file or directory,[cat: goodbye.txt: No such file or directory],cat: $: No such file or directory
411,2022-09-11 20:13:00,User_3,cat,cat hello.txt,,[],None
412,2022-09-11 20:20:00,User_3,cat,cat,,[],None
413,2022-09-11 20:22:00,User_3,ls,ls > ls.out,,[],None


Error results because type the wrong file name. a Typo

**Another Example: ls: cannot access $: No such file or directory**

In [22]:
error_filter_ls = (error_expand["no name"] == "ls: cannot access $: No such file or directory")

In [23]:
ls_error = error_expand[error_filter_ls]
ls_error.head(10)

,time,User Name,command,full_command,stderr,error list,no name
58,2021-04-16 22:56:00,User_9,ls,ls -l misch_child/goodbye.txt,ls: cannot access 'misch_child/goodbye.txt': N...,[ls: cannot access 'misch_child/goodbye.txt': ...,ls: cannot access $: No such file or directory
264,2022-09-11 18:27:00,User_3,ls,ls /home/jovyan/RNA_SEQ_SAMPLES,ls: cannot access '/home/jovyan/RNA_SEQ_SAMPLE...,[ls: cannot access '/home/jovyan/RNA_SEQ_SAMPL...,ls: cannot access $: No such file or directory
270,2022-09-11 18:28:00,User_3,ls,ls /gunings/RNA_SEQ_SAMPLES,ls: cannot access '/gunings/RNA_SEQ_SAMPLES': ...,[ls: cannot access '/gunings/RNA_SEQ_SAMPLES':...,ls: cannot access $: No such file or directory
285,2022-09-11 18:33:00,User_3,ls,"ls stdout,txt.","ls: cannot access 'stdout,txt.': No such file ...","[ls: cannot access 'stdout,txt.': No such file...",ls: cannot access $: No such file or directory
286,2022-09-11 18:35:00,User_3,ls,ls RNAt,ls: cannot access 'RNAt': No such file or dire...,[ls: cannot access 'RNAt': No such file or dir...,ls: cannot access $: No such file or directory
287,2022-09-11 18:37:00,User_3,ls,ls RNA,ls: cannot access 'RNA': No such file or direc...,[ls: cannot access 'RNA': No such file or dire...,ls: cannot access $: No such file or directory
288,2022-09-11 18:37:00,User_3,ls,ls RNA^t,ls: cannot access 'RNA^t': No such file or dir...,[ls: cannot access 'RNA^t': No such file or di...,ls: cannot access $: No such file or directory
290,2022-09-11 18:51:00,User_3,ls,ls RNA^t,ls: cannot access 'RNA^t': No such file or dir...,[ls: cannot access 'RNA^t': No such file or di...,ls: cannot access $: No such file or directory
299,2022-09-11 18:57:00,User_3,ls,ls –l,ls: cannot access '–l': No such file or directory,[ls: cannot access '–l': No such file or direc...,ls: cannot access $: No such file or directory
302,2022-09-11 19:17:00,User_3,ls,ls –l stdout.txt,ls: cannot access '–l': No such file or directory,[ls: cannot access '–l': No such file or direc...,ls: cannot access $: No such file or directory


**ls example: 58**

In [24]:
ls_error_58 = error_expand.iloc[55:62]
ls_error_58

,time,User Name,command,full_command,stderr,error list,no name
55,2021-04-16 22:56:00,User_9,cd,cd ..,,[],None
56,2021-04-16 22:56:00,User_9,ls,ls,,[],None
57,2021-04-16 22:56:00,User_9,ln,ln -s goodbye.txt misc_child,,[],None
58,2021-04-16 22:56:00,User_9,ls,ls -l misch_child/goodbye.txt,ls: cannot access 'misch_child/goodbye.txt': N...,[ls: cannot access 'misch_child/goodbye.txt': ...,ls: cannot access $: No such file or directory
59,2021-04-16 22:56:00,User_9,ls,ls,,[],None
60,2021-04-16 22:57:00,User_9,ls,ls -l misc_child,,[],None
61,2021-04-16 23:05:00,User_9,cp,cp goodbye.txt copied_goodbye.txt,,[],None


Reason of error: type wrong path name

**ls example 264**

In [26]:
ls_error_264 = error_expand.iloc[261:270]
ls_error_264

,time,User Name,command,full_command,stderr,error list,no name
261,2022-03-02 20:57:00,User_8,ls,ls /home/jovyan/,,[],None
262,2022-03-02 20:58:00,User_8,ls,ls /home/jovyan/data,,[],None
263,2022-09-11 18:18:00,User_3,ls,ls /home/jovyan,,[],None
264,2022-09-11 18:27:00,User_3,ls,ls /home/jovyan/RNA_SEQ_SAMPLES,ls: cannot access '/home/jovyan/RNA_SEQ_SAMPLE...,[ls: cannot access '/home/jovyan/RNA_SEQ_SAMPL...,ls: cannot access $: No such file or directory
265,2022-09-11 18:27:00,User_3,ls,ls ..,,[],None
266,2022-09-11 18:27:00,User_3,ls,ls,,[],None
267,2022-09-11 18:28:00,User_3,ls,ls,,[],None
268,2022-09-11 18:28:00,User_3,cd,cd home,,[],None
269,2022-09-11 18:28:00,User_3,ls,ls,,[],None


Reason of error: wrong path

**ls example 285**

In [27]:
ls_error_285 = error_expand.iloc[282:290]
ls_error_285

,time,User Name,command,full_command,stderr,error list,no name
282,2022-09-11 18:32:00,User_3,cd,cd ..,,[],None
283,2022-09-11 18:32:00,User_3,ls,ls,,[],None
284,2022-09-11 18:33:00,User_3,ls,ls stdout.txt,,[],None
285,2022-09-11 18:33:00,User_3,ls,"ls stdout,txt.","ls: cannot access 'stdout,txt.': No such file ...","[ls: cannot access 'stdout,txt.': No such file...",ls: cannot access $: No such file or directory
286,2022-09-11 18:35:00,User_3,ls,ls RNAt,ls: cannot access 'RNAt': No such file or dire...,[ls: cannot access 'RNAt': No such file or dir...,ls: cannot access $: No such file or directory
287,2022-09-11 18:37:00,User_3,ls,ls RNA,ls: cannot access 'RNA': No such file or direc...,[ls: cannot access 'RNA': No such file or dire...,ls: cannot access $: No such file or directory
288,2022-09-11 18:37:00,User_3,ls,ls RNA^t,ls: cannot access 'RNA^t': No such file or dir...,[ls: cannot access 'RNA^t': No such file or di...,ls: cannot access $: No such file or directory
289,2022-09-11 18:51:00,User_3,cd,cd home,,[],None


Reason of error: typo of file name. Should be period but use comma

**ls example 302**

In [28]:
ls_error_302 = error_expand.iloc[299:306]
ls_error_302

,time,User Name,command,full_command,stderr,error list,no name
299,2022-09-11 18:57:00,User_3,ls,ls –l,ls: cannot access '–l': No such file or directory,[ls: cannot access '–l': No such file or direc...,ls: cannot access $: No such file or directory
300,2022-09-11 18:57:00,User_3,ls,ls -l,,[],None
301,2022-09-11 19:17:00,User_3,ls,ls -l stdout.txt,,[],None
302,2022-09-11 19:17:00,User_3,ls,ls –l stdout.txt,ls: cannot access '–l': No such file or directory,[ls: cannot access '–l': No such file or direc...,ls: cannot access $: No such file or directory
303,2022-09-11 19:17:00,User_3,ls,ls,,[],None
304,2022-09-11 19:17:00,User_3,cd,cd ..,,[],None
305,2022-09-11 19:17:00,User_3,ls,ls,,[],None


Error reason: cannot link